# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [20]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [21]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [22]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [23]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [24]:
from cassandra.cluster import Cluster
"""
    We create the connection by passing the local host address to the Cluster function from Cassandra.
"""
cluster = Cluster(['127.0.0.1'])

session = cluster.connect()

#### Create Keyspace

In [25]:
"""
    We create the keyspace using the function execute and passing the parameters as string.
"""
try:
    session.execute("""
            CREATE KEYSPACE IF NOT EXISTS udacity
            WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [26]:
"""
    We set the keyspace using the function  set_keyspace in session.
"""
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [27]:
"""
    We create the create statement for each of the tables assigning the columns that each table will 
    contain and what will be the combination of partition key and clustering columns of each table.
"""
songs_play = "CREATE TABLE IF NOT EXISTS songs_play"
songs_play = songs_play + "(sessionID int, itemInSession int, artist text, song text, length decimal, PRIMARY KEY(sessionID, itemInSession))"
try:
    session.execute(songs_play)
except Exception as e:
    print(e)

user_artists = "CREATE TABLE IF NOT EXISTS user_artists"
user_artists = user_artists + "(userid int, sessionID int, itemInSession int, artist text, song text, first_name text, last_name text, PRIMARY KEY(userid, sessionID, itemInSession))"
try:
    session.execute(user_artists)
except Exception as e:
    print(e)

song_history = "CREATE TABLE IF NOT EXISTS song_history"
song_history = song_history + "(song text, userid int, artist text, first_name text, last_name text, PRIMARY KEY(song, userid))"
try:
    session.execute(song_history)
except Exception as e:
    print(e)



In [28]:
"""
    We creates the insert querys for songs_play, user_artists and song_history.
"""
file = 'event_datafile_new.csv'

songs_play_insert = "INSERT INTO songs_play (sessionID, itemInSession, artist, song, length)"
songs_play_insert = songs_play_insert + "VALUES(%s, %s, %s, %s, %s)"

user_artists_insert = "INSERT INTO user_artists (userid, sessionID, itemInSession, artist, song, first_name, last_name)"
user_artists_insert = user_artists_insert + "VALUES(%s, %s, %s, %s, %s, %s, %s)"

song_history_insert = "INSERT INTO song_history (song, artist, first_name, last_name, userid)"
song_history_insert = song_history_insert + "VALUES(%s, %s, %s, %s, %s)"


"""
    1 - We open the file event_datafile_new.csv se ecnoding to utf8.
    2 - We pass the file to the function csv.reader (f) that returns a reader object.
    3 - We skip the header
    4 - We iterate through each line of the file
    5 - For each of the tables using the execute function, we pass as a parameter the query that we will use to insert and with the index corresponding to each column and we pass it to the list to obtain the value that we are going to insert in each column.
"""
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(songs_play_insert, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        session.execute(user_artists_insert, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        session.execute(song_history_insert, (line[9], line[0], line[1], line[4], int(line[10])))



#### Do a SELECT to verify that the data have been inserted into each table

In [30]:
"""
    We select artist, song and song length for song listened in sessionID = 338 and itemInSession = 4.
"""

query = "SELECT artist, song, length FROM songs_play WHERE sessionId = 338 AND itemInSession = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [31]:
"""
    We select artist, song and user first_name and last_name for user whos userid is 182 and has a sessionid = 182.  
"""

query = "SELECT iteminsession, artist, song, first_name, last_name  FROM user_artists WHERE  userid = 10 and sessionid = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
     
for row in rows:
    print (row.iteminsession, row.artist, row.song, row.first_name, row.last_name)

                    

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [32]:
"""
    We select ervery users who listened to the 'All Hands Against His Own' song and print the user first_name and last_name.
"""

query = "SELECT first_name, last_name FROM song_history WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.first_name, row.last_name)

                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [33]:
"""
    We drop tables songs_play, user_artists, song_history if it exists.
"""
drop_songs_play = "DROP TABLE IF EXISTS songs_play"
try:
    session.execute(drop_songs_play)
except Exception as e:
    print(e)

drop_user_artists = "DROP TABLE IF EXISTS user_artists"
try:
    session.execute(drop_user_artists)
except Exception as e:
    print(e)

drop_song_history = "DROP TABLE IF EXISTS song_history"
try:
    session.execute(drop_song_history)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [34]:
session.shutdown()
cluster.shutdown()